In [1]:
# | results: hide
import pandas as pd
from datetime import datetime
import requests
import holoviews as hv
import locale
import numpy as np

locale.setlocale(locale.LC_TIME, "de_DE.UTF-8")
hv.extension("bokeh", logo=False)

In [2]:
YEAR = 2025
VAT = 1.19
TADO_FIXED_FEE_PER_KWH = 0.21540  # EUR/kWh
AEUW_FEE_PER_KWH = 0.3462  # EUR/kWh
BASE_FEE_AUEW = 15.68

# for optimized use of storage system
BATTERY_CAPACITY = 22_000 * 0.8  # Wh, 20% are not usable (reserve)
ROUNDTRIP_EFFICIENCY = 0.9  # assuming 90% efficiency when charing and discharging the same day
MAX_CHARGING_POWER = 4000  # Watt, this is the max power the Battery system can charge with
MIN_PRICEDIFF = 0.06  # EUR / kWh, do not charge storage below this threshold

In [3]:
# Get hourly prices from awattar API
begin = datetime.fromisoformat("2020-01-01").timestamp()
end = datetime.now().timestamp()
# API endpoint URL
url = f"https://api.awattar.de/v1/marketdata?start={int(begin) * 1000}&end={int(end) * 1000}"

# Fetch JSON data from the API
response = requests.get(url)
data = response.json()

# Convert JSON data to Pandas DataFrame
hourly_price = pd.DataFrame(data["data"])

hourly_price["start"] = pd.to_datetime(hourly_price["start_timestamp"], unit="ms")
hourly_price["end"] = pd.to_datetime(hourly_price["end_timestamp"], unit="ms")
hourly_price["marketprice"], hourly_price["unit"] = hourly_price["marketprice"] / 1000, "EUR/kWh"
hourly_price["start_day"] = hourly_price["start"].dt.strftime(
    "%m-%d %H:%M:%S"
)  # day without year (for inter-year comparison)
hourly_price["day"] = hourly_price["start"].dt.strftime("%Y-%m-%d")
hourly_price["real_price"] = hourly_price["marketprice"] * VAT + TADO_FIXED_FEE_PER_KWH

In [4]:
# Get recorded interval consumption and production data
all_interval = pd.read_csv("../data/2025_messwerte_pseudonymisiert_3600s_interval.csv", parse_dates=["time"])

In [5]:
consumption = (
    all_interval.loc[lambda x: x["name"].isin(["Wohnung 1", "Wohnung 2", "Wohnung 3", "Wohnung 4"])]
    .groupby("time")
    .apply(lambda y: y.assign(Wh_total=lambda x: x["Wh"].sum()))
    .reset_index(drop=True)
)
production = (
    all_interval.loc[
        lambda x: x["name"].isin(
            ["Sunny_Island_Batterie_entladen", "Sunny_Island_Netzbezug", "Sunny_Tripower_Gesamtertrag"]
        )
    ]
    .rename(columns={"Wh": "Wh_prod"})
    .groupby("time")
    .apply(lambda y: y.assign(Wh_total_prod=lambda x: x["Wh_prod"].sum())).reset_index(drop=True)
)

/tmp/ipykernel_118891/3164754718.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda y: y.assign(Wh_total=lambda x: x["Wh"].sum()))
/tmp/ipykernel_118891/3164754718.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda y: y.assign(Wh_total_prod=lambda x: x["Wh_prod"].sum())).reset_index(drop=True)


In [6]:
production

,name,time,Wh_prod,Wh_total_prod
0,Sunny_Island_Batterie_entladen,2025-01-01 00:00:00,0.00000,860.000000
1,Sunny_Island_Netzbezug,2025-01-01 00:00:00,860.00000,860.000000
2,Sunny_Tripower_Gesamtertrag,2025-01-01 00:00:00,0.00000,860.000000
3,Sunny_Island_Batterie_entladen,2025-01-01 01:00:00,0.00000,680.000000
4,Sunny_Island_Netzbezug,2025-01-01 01:00:00,680.00000,680.000000
...,...,...,...,...
26274,Sunny_Island_Netzbezug,2025-12-31 22:00:00,10.00000,1008.988194
26275,Sunny_Tripower_Gesamtertrag,2025-12-31 22:00:00,0.00000,1008.988194
26276,Sunny_Island_Batterie_entladen,2025-12-31 23:00:00,879.75625,899.756250
26277,Sunny_Island_Netzbezug,2025-12-31 23:00:00,20.00000,899.756250


In [7]:
# Calculate cost per hour, considering only the fraction that was drawn from the grid
cost_df = (
    consumption.merge(production.loc[lambda x: x["name"] == "Sunny_Island_Netzbezug"].drop(columns=["name"]), on="time")
    .merge(hourly_price, left_on="time", right_on="start")
    # Wh/Wh_total gives the current fraction of the full energy consumption of a given flat. 
    # Multiplied with Wh_prod (=grid usage) it gives the current share of grid usage
    .assign(Wh_grid=lambda x: (x["Wh"] / x["Wh_total"] * x["Wh_prod"])) 
    .assign(
        # only energy from grid
        price_tado=lambda x: x["Wh_grid"] * x["real_price"] / 1000,
        price_auew=lambda x: x["Wh_grid"] * AEUW_FEE_PER_KWH / 1000,
        # all energy, theoretically, for comparison
        price_tado_no_pv=lambda x: x["Wh"] * x["real_price"] / 1000,
        price_auew_no_pv=lambda x: x["Wh"] * AEUW_FEE_PER_KWH / 1000,
    )
)

In [8]:
# Aggregate by Month and year
cost_per_month = (
    cost_df.groupby("name")
    .apply(
        lambda x: x.set_index("time")[
            ["Wh", "Wh_grid", "price_tado", "price_auew", "price_tado_no_pv", "price_auew_no_pv"]
        ]
        .resample("MS")
        .sum()
    )
    .reset_index()
    .assign(month=lambda x: pd.to_datetime(x["time"]).dt.strftime("%b"))
    .assign(month=lambda x: pd.Categorical(x["month"], categories=x["month"].unique()))
)

cost_per_year = cost_df.groupby("name")[
    ["Wh", "Wh_grid", "price_tado", "price_auew", "price_tado_no_pv", "price_auew_no_pv"]
].sum()

/tmp/ipykernel_118891/38339815.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


In [9]:
cost_df.assign(day=lambda x: x["time"].dt.strftime("%Y-%m-%d")).drop(columns=["time", "unit", "start", "end"]).groupby(["day", "name"]).sum().reset_index()

,day,name,Wh,Wh_total,Wh_prod,Wh_total_prod,start_timestamp,end_timestamp,marketprice,start_day,real_price,Wh_grid,price_tado,price_auew,price_tado_no_pv,price_auew_no_pv
0,2025-01-01,Wohnung 1,7242.0,20592.0,5100.0,36909.139583,41657544000000,41657630400000,0.02330,01-01 00:00:0001-01 01:00:0001-01 02:00:0001-0...,5.197327,2271.072471,0.489938,0.786245,1.566886,2.507180
1,2025-01-01,Wohnung 2,6985.0,20592.0,5100.0,36909.139583,41657544000000,41657630400000,0.02330,01-01 00:00:0001-01 01:00:0001-01 02:00:0001-0...,5.197327,2165.811095,0.467418,0.749804,1.512996,2.418207
2,2025-01-01,Wohnung 3,6041.0,20592.0,5100.0,36909.139583,41657544000000,41657630400000,0.02330,01-01 00:00:0001-01 01:00:0001-01 02:00:0001-0...,5.197327,542.027617,0.117025,0.187650,1.304168,2.091394
3,2025-01-01,Wohnung 4,324.0,20592.0,5100.0,36909.139583,41657544000000,41657630400000,0.02330,01-01 00:00:0001-01 01:00:0001-01 02:00:0001-0...,5.197327,121.088817,0.026299,0.041921,0.070039,0.112169
4,2025-01-02,Wohnung 1,9924.0,35866.0,16350.0,36209.232639,41659617600000,41659704000000,2.32684,01-02 00:00:0001-02 01:00:0001-02 02:00:0001-0...,7.938540,4054.706735,1.463247,1.403739,3.356046,3.435689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,2025-12-30,Wohnung 4,26.0,21747.0,10040.0,28350.104861,42410260800000,42410347200000,2.11650,12-30 00:00:0012-30 01:00:0012-30 02:00:0012-3...,7.688235,2.822478,0.000896,0.000977,0.008256,0.009001
1456,2025-12-31,Wohnung 1,9362.0,27385.0,6710.0,48753.966667,42412334400000,42412420800000,2.06574,12-31 00:00:0012-31 01:00:0012-31 02:00:0012-3...,7.627831,3206.966446,1.005018,1.110252,2.983257,3.241124
1457,2025-12-31,Wohnung 2,6150.0,27385.0,6710.0,48753.966667,42412334400000,42412420800000,2.06574,12-31 00:00:0012-31 01:00:0012-31 02:00:0012-3...,7.627831,2473.515663,0.777436,0.856331,1.964332,2.129130
1458,2025-12-31,Wohnung 3,11872.0,27385.0,6710.0,48753.966667,42412334400000,42412420800000,2.06574,12-31 00:00:0012-31 01:00:0012-31 02:00:0012-3...,7.627831,1029.504611,0.326414,0.356414,3.830204,4.110086


In [10]:
cost_per_year

,Wh,Wh_grid,price_tado,price_auew,price_tado_no_pv,price_auew_no_pv
name,,,,,,
Wohnung 1,3736778.0,656635.768490,226.908017,227.327303,1205.485608,1293.672544
Wohnung 2,1803410.0,594103.989939,206.341143,205.678801,611.194930,624.340542
Wohnung 3,2018200.0,344554.905926,125.553608,119.284908,665.584706,698.700840
Wohnung 4,588434.0,82513.313354,29.987210,28.566109,191.053284,203.715851


In [11]:
def optimize_cost_using_storage(df: pd.DataFrame, debug: bool = False):
    """For a given day, optimize cost by using the battery storage

    This function simulates a simple cost optimization using a battery storage and a dynamic electricity tariff.

    Given a data frame with
     * hour
     * price for that hour
     * battery discharge per hour
     * pv dirrect use per hour
     * grid usage per hour

    It generates a new data frame with
     * simulated battery charge per hour
     * simulated grid usage per hour

    It is not fully realistic, as we have the information about real consumption which is
    not available yet when we need to predict how much to charge. It also makes the simplifying assumption
    that the storage system is always empty at midnight and we optimize power for the next day.

    So not perfect, but it should give us a reasonable estimate of what margin of cost saving we could
    realistically expect.
    """
    df = df.set_index("time").sort_index()
    if debug:
        print(f"Working on day {df.index[0]}")
    hourly_price = df.loc[lambda x: x["name"] == "Sunny_Island_Batterie_entladen", "real_price"].values
    battery_discharge = df.loc[lambda x: x["name"] == "Sunny_Island_Batterie_entladen", "Wh_prod"].values
    battery_discharge_optimized = np.copy(battery_discharge)
    pv_production = df.loc[lambda x: x["name"] == "Sunny_Tripower_Gesamtertrag", "Wh_prod"].values
    grid_usage = df.loc[lambda x: x["name"] == "Sunny_Island_Netzbezug", "Wh_prod"].values
    grid_usage_optimized = np.copy(grid_usage)
    battery_charge_optimized = np.zeros(hourly_price.shape)
    cheapest_hours = np.argsort(hourly_price)
    most_expensive_hours = cheapest_hours[::-1]

    if (
        not len(hourly_price)
        == len(battery_discharge_optimized)
        == len(pv_production)
        == len(grid_usage_optimized)
        == 24
    ):
        print(f"Warning: day {df.index[0]} does not have exactly 24h of recorded data. Not performing optimization.")
        return pd.DataFrame()

    for most_expensive_hour_idx in most_expensive_hours:
        if debug:
            print(f"Most expensive_hour: {most_expensive_hour_idx}")
        hour_needs_charge = grid_usage_optimized[most_expensive_hour_idx] / ROUNDTRIP_EFFICIENCY
        while hour_needs_charge > 0:
            (available_hours_for_charge_idxs,) = np.where(
                (cheapest_hours < most_expensive_hour_idx) & (battery_charge_optimized < MAX_CHARGING_POWER)
            )
            if not len(available_hours_for_charge_idxs):
                if debug:
                    print("can't further optimize this hour")
                # can't further optimize this hour
                break
            cheapest_hour_available_for_charge_idx = cheapest_hours[
                np.isin(cheapest_hours, available_hours_for_charge_idxs)
            ][0]
            pricediff = hourly_price[most_expensive_hour_idx] - hourly_price[cheapest_hour_available_for_charge_idx]
            if pricediff < MIN_PRICEDIFF:
                if debug:
                    print("not worth optmizing")
                # not worth optmiizing
                break
            can_charge = np.min(
                [
                    MAX_CHARGING_POWER - battery_charge_optimized[cheapest_hour_available_for_charge_idx],
                    hour_needs_charge,
                ]
            )
            battery_charge_optimized[cheapest_hour_available_for_charge_idx] += can_charge
            grid_usage_optimized[most_expensive_hour_idx] -= can_charge * ROUNDTRIP_EFFICIENCY
            battery_discharge_optimized[most_expensive_hour_idx] += can_charge * ROUNDTRIP_EFFICIENCY
            hour_needs_charge -= can_charge

    return pd.DataFrame().assign(
        time=df.index.drop_duplicates(),
        hourly_price=hourly_price,
        battery_discharge_optimized=battery_discharge_optimized,
        battery_discharge=battery_discharge,
        pv_production=pv_production,
        grid_usage_optimized=grid_usage_optimized + battery_charge_optimized,
        grid_usage=grid_usage,
        battery_charge_optimized=battery_charge_optimized,
    )

In [12]:
production_optimized_per_day = (
    production.merge(hourly_price, left_on="time", right_on="start")
    .groupby("day")
    .apply(optimize_cost_using_storage, debug=False)
    .reset_index(drop=True)
    .assign(
        total_prod_optimized=lambda x: x["grid_usage_optimized"] + x["pv_production"] + x["battery_discharge_optimized"],
        total_prod=lambda x: x["grid_usage"] + x["pv_production"] + x["battery_discharge"],
    ).assign(
        price_tado_optimized=lambda x: x["grid_usage_optimized"] * x["hourly_price"] / 1000,
        price_tado=lambda x: x["grid_usage"] * x["hourly_price"] / 1000,
        price_auew=lambda x: x["grid_usage"] * AEUW_FEE_PER_KWH / 1000,
        day=lambda x: x["time"].dt.strftime("%Y-%m-%d"),
    )
    .drop(columns=["time"])
    .groupby("day")
    .sum()
)

consumption_per_day = (
    consumption.assign(day=lambda x: x["time"].dt.strftime("%Y-%m-%d"))
    .drop(columns="time")
    .groupby(["day", "name"])
    .sum()
    .reset_index()
    .groupby("day")
    .apply(lambda x: x.assign(Wh_total=lambda y: y["Wh"].sum()))
    .reset_index(drop=True)
)

cost_df_optimized = production_optimized_per_day.merge(consumption_per_day, on="day").assign(
    price_auew_per_flat = lambda x: x["price_auew"] * (x["Wh"] / x["Wh_total"]), 
    price_tado_per_flat = lambda x: x["price_tado"] * (x["Wh"] / x["Wh_total"]),
    price_tado_optimized_per_flat=lambda x: x["price_tado_optimized"] * (x["Wh"] / x["Wh_total"]),
)

/tmp/ipykernel_118891/1823218581.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(optimize_cost_using_storage, debug=False)
/tmp/ipykernel_118891/1823218581.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(Wh_total=lambda y: y["Wh"].sum()))


In [13]:
cost_df_optimized.groupby("name")[
    ["Wh", "price_auew_per_flat", "price_tado_per_flat", "price_tado_optimized_per_flat"]
].sum().assign(pct_saving = lambda x: (1 - (x["price_tado_optimized_per_flat"] / x["price_tado_per_flat"])) * 100)

,Wh,price_auew_per_flat,price_tado_per_flat,price_tado_optimized_per_flat,pct_saving
name,,,,,
Wohnung 1,3726998.0,236.076723,237.468592,220.539427,7.129012
Wohnung 2,1801831.0,190.272326,192.995540,180.168443,6.646318
Wohnung 3,2011902.0,137.261675,139.824996,129.035545,7.716397
Wohnung 4,578569.0,27.162017,27.472314,23.759747,13.513847


# Dynamischer Stromtarif vs. Fixpreistarif in Kombination mit Photovoltaikanlage

In unserer Eigentümergemeinschaft kann ich den Stromverbrauch der vier Wohneinheiten [digital in Echtzeit auslesen](https://github.com/grst/energymeter). Bereits im Vergangenen Jahr habe ich anhand der Verbrauchsdaten von 2024 unseren momentanen Stromtarif mit [Tado Hourly](https://energy.tado.com/) verglichen. Dabei kam ich zu dem Ergebnis, dass der dynamische Stromtarif auch ohne steuerbare Verbraucher  für alle Wohneinheiten (geringfügig) günstiger ist. 

In diesem Post gibt es ein Update mit den Daten von 2025. Zusätzlich berücksichtige ich in diesem Jahr den Strom, der von der Photovolatik (PV)-anlage produziert wurde. Möglicherweise ist in Kombination mit PV der dynamische Stromtarif gar nicht mehr günstiger, weil der Strom vermutlich meist dann im Überfluss vorhanden ist, wenn man selber gerade Strom vom eigenen Dach bezieht. 

Die Rohdaten und das Python Notebook mit der Analyse stehen [auf GitHub zur Verfügung](https://github.com/grst/dynamischer-stromtarif). 

## Die Tarife

Es gibt eine Unzahl an unterschiedlichen Stromtarifen in Deutschland mit teilweise sehr unterschiedlichen
Preisstrukturen. Hier betrachte ich nur zwei Tarife: 

 * [AÜW Allgäustrom Basis](https://auew.de/privatkunden/strom/allgaeustrom-basis/), unser bisheriger Anbieter, Preisniveau für 2025
 * [Tado Hourly][]

Die Tarife gestalten sich wie folgt (alle Preise inkl. MwSt):

| Tarif | Arbeitspreis pro kWh | Grundpreis pro Monat |
| -- | -- | -- |
| AÜW | 34,62 ct (Strompreis + Steuern + Netzgebühr) | 15,68 EUR |
| tado | Epex Spot Day Ahead <br> + 19% MwSt <br> + 1,785ct (Aufschlag) <br> + 21,540 ct (Netzgebühr + Steuern) | 16,60 EUR |

: Verglichene Stromtarife {.striped .hover tbl-colwidths="[25,50,25]"}


Die Grundpreise sind sehr ähnlich, daher betrachte ich nur die Arbeitspreise. 

[Tado Hourly]: https://energy.tado.com/
[Tibber]: https://tibber.com/de

## Die Wohneinheiten

| Wohnung | Beschreibung |
| -- | -- |
|Wohnung 1 | Ganzjährig bewohnt von einer alleinstehenden Person. Ca. 25 Jahre alte Haushaltsgeräte. Aus historischen Gründen hängen diverse gemeinschaftliche Verbraucher (Treppenhaus, Heizung, ...) zusätzlich auf diesen Zähler |
|Wohnung 2 | Ganzjährig bewohnt von zwei Personen. Teilweise erneuerte Haushaltsgeräte. Eigene 3kWp Photovolatikanlage mit 6kWh Speicher hinter dem Zähler. |
|Wohnung 3 | Ganzjährig bewohnt von einer Familie mit zwei kleinen Kindern. Neubauwohnung mit modernen Haushaltsgeräten. |
|Wohnung 4 | Sporadisch genutzte Ferienwohnung |

: Verbrauchsprofile vier Wohnungen {.striped .hover tbl-colwidths="[25,75]"}

## Die PV Analage

Wir haben eine 24 kWp PV Analge mit 22 kWh Stromspeicher, die von allen Wohneinheiten gemeinsam genutzt wird. Anlage und Speicher sind so dimensioniert dass wir an einem sonnigen, schneefreien Dezembertag zu 100% autark sind. Insgesamt erreichten wir einen im Jahr 2025 einen Autarkiegrad von 83%. 

## Stromkosten 2025 -- ohne PV

Was hätten die jeweiligen Wohneinheiten 2025 bezahlt, wenn keine PV Anlage installierte wäre?

In [14]:
# | tbl-cap: Jährliche Stromkosten in EUR für AÜR und Tado pro Wohneinheit (ohne PV-Anlage)

cost_per_year.reset_index().rename(
    columns={"name": "Wohneinheit", "price_tado_no_pv": "Tado (EUR)", "price_auew_no_pv": "AÜW (EUR)"}
).assign(
    **{
        "Verbrauch (kWh)": lambda x: x["Wh"] / 1000,
        "Ersparnis (EUR)": lambda x: x["AÜW (EUR)"] - x["Tado (EUR)"],
        "Ersparnis (%)": lambda x: (1 - x["Tado (EUR)"] / x["AÜW (EUR)"]) * 100,
    }
).set_index("Wohneinheit")[
    ["Verbrauch (kWh)", "Tado (EUR)", "AÜW (EUR)", "Ersparnis (EUR)", "Ersparnis (%)"]
].style.format("{:.2f}")

,Verbrauch (kWh),Tado (EUR),AÜW (EUR),Ersparnis (EUR),Ersparnis (%)
Wohneinheit,,,,,
Wohnung 1,3736.78,1205.49,1293.67,88.19,6.82
Wohnung 2,1803.41,611.19,624.34,13.15,2.11
Wohnung 3,2018.20,665.58,698.70,33.12,4.74
Wohnung 4,588.43,191.05,203.72,12.66,6.22


Die folgende Grafik zeigt den Vergleich monatsweise:

In [15]:
hv.Table(
    cost_per_month.rename(
        columns={"name": "Wohneinheit", "price_tado_no_pv": "Tado", "price_auew_no_pv": "AÜW", "month": "Monat"}
    )[["Wohneinheit", "Monat", "Tado", "AÜW"]].melt(
        ["Wohneinheit", "Monat"], var_name="Tarif", value_name="Kosten (EUR)"
    )
).to.bars(["Monat", "Tarif"], "Kosten (EUR)").opts(width=450, xrotation=90, legend_position="right")

:HoloMap   [Wohneinheit]
   :Bars   [Monat,Tarif]   (Kosten (EUR))

[Wie bereits 2024](https://grst.github.io/dynamischer-stromtarif/dynamischer-stromtarif.html#vergleich-f%C3%BCr-2024) wären die Kosten des dynamischen Stromtarifs geringer als mit dem Fixpreistarif. Allerdings sind die Einsparungen mit 1,9 - 6,8% geringer als im Vorjahr (10-13%). Das könnte unter anderem daran liegen, dass der fixe Anteil pro kWh (Netzgebühren + Steuern) 2025 mit 21,54 ct/kWh nochmals deutlich höher liegt als im Vorjahr (19,116 ct/kWh). 

## Stromkosten 2025 -- mit PV

Nun derselbe Vergleich, allerdings unter Berücksichtigung der PV-Anlage. D.h. bei der Berechnung wurde für jedes 
Stundenintervall nur der Anteil berücksichtigt, der aus dem Stromnetz bezogen wurde. 

In [16]:
# | tbl-cap: Jährliche Stromkosten in EUR für AÜR und Tado pro Wohneinheit (mit PV Anlage)

cost_per_year.reset_index().rename(
    columns={"name": "Wohneinheit", "price_tado": "Tado (EUR)", "price_auew": "AÜW (EUR)"}
).assign(
    **{
        "Verbrauch (kWh)": lambda x: x["Wh"] / 1000,
        "davon Netzbezug (%)": lambda x: x["Wh_grid"] / x["Wh"] * 100,
        "Ersparnis (EUR)": lambda x: x["AÜW (EUR)"] - x["Tado (EUR)"],
        "Ersparnis (%)": lambda x: (1 - x["Tado (EUR)"] / x["AÜW (EUR)"]) * 100,
    }
).set_index("Wohneinheit")[
    ["Verbrauch (kWh)", "davon Netzbezug (%)", "Tado (EUR)", "AÜW (EUR)", "Ersparnis (EUR)", "Ersparnis (%)"]
].style.format("{:.2f}")

,Verbrauch (kWh),davon Netzbezug (%),Tado (EUR),AÜW (EUR),Ersparnis (EUR),Ersparnis (%)
Wohneinheit,,,,,,
Wohnung 1,3736.78,17.57,226.91,227.33,0.42,0.18
Wohnung 2,1803.41,32.94,206.34,205.68,-0.66,-0.32
Wohnung 3,2018.20,17.07,125.55,119.28,-6.27,-5.26
Wohnung 4,588.43,14.02,29.99,28.57,-1.42,-4.97


Die folgende Grafik zeigt den Vergleich monatsweise: 

In [17]:
hv.Table(
    cost_per_month.rename(columns={"name": "Wohneinheit", "price_tado": "Tado", "price_auew": "AÜW", "month": "Monat"})[
        ["Wohneinheit", "Monat", "Tado", "AÜW"]
    ].melt(["Wohneinheit", "Monat"], var_name="Tarif", value_name="Kosten (EUR)")
).to.bars(["Monat", "Tarif"], "Kosten (EUR)").opts(width=450, xrotation=90, legend_position="right")

:HoloMap   [Wohneinheit]
   :Bars   [Monat,Tarif]   (Kosten (EUR))

In Kombination mit PV Anlage ist der dynamische Stromtarif aufs Jahr 2025 gerechnet also um wenige Euro *teurer* als unser
momentaner Fixpreistarif. Dies liegt daran, dass mit PV-Anlage und Stromspeicher hauptsächlich Strom in den Wintermonaten
bezogen wird, in denen der durchschnittliche Börsenpreis höher ist. 

## Fazit

Ohne steuerbare Großverbraucher (wie Wärmepumpe oder E-Auto) ist für einen normalen Haushalt der finanzielle Anreiz
auf einen dynamischen Stromtarif zu wechseln gering (Ersparnis von wenigen Euro pro Monat). Mit PV Anlage und Stromspeicher ist der Anreiz nicht vorhanden (Mehrkosten von wenigen Euro pro Jahr). 

Interessant wäre in Kombination mit einem dynamischen Stromtarif, den Stromspeicher im Winter mit Netzstrom zu laden wenn 
er gerade günstig ist (z.B. nachts). Das Einsparpotenzial bedarf einer separaten Analyse. 
